In [32]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from model.reasoning import ConnectionsDataModule
from trl_wrapper.trainer_wrapper import TrainerWrapper, GRPO_MATH_CONFIG, GRPO_CONNECTIONS_CONFIG
from trl_wrapper.wrapper_config import DatasetConfig

wrapper = TrainerWrapper(GRPO_CONNECTIONS_CONFIG)

wrapper.init_data_module()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2025-03-26 20:52:59.935 | INFO     | trl_wrapper.trainer_wrapper:__init__:266 - Setting padding side to: left
2025-03-26 20:52:59.950 | INFO     | trl_wrapper.wrapper_config:__init__:133 - Cache dir: ../dataset_caches/connections_data_module
Map: 100%|██████████| 353/353 [00:00<00:00, 15763.86 examples/s]

Dataset({
    features: ['words', 'solution', 'prompt', 'answer'],
    num_rows: 317
})


In [44]:
# sample = data_module.train_dataset[0]
# sample
wrapper.data_module.train_dataset
first_sample = wrapper.data_module.train_dataset[0]

conv = first_sample['prompt']
fake_thoughts = "<thought> First four are academic</thought> <thought> Last two are not academic</thought>"
conv = [*conv, {'role': "assistant", 'content': first_sample['answer']}]
conv

[{'content': '\nYou are an expert puzzle solving model.\nFind groups of words that are related to each other, and return the answer in the following format:\nRespond in the following format:\n<reasoning>\n...\n</reasoning>\n<answer>\n<group>\n...\n</group>\n<group>\n...\n</group>\n</answer>\n\n# Example\n\nUser: apple, orange, banana, pear, corolla, charger,\nAssistant: <reasoning>\nThe first group are all fruits.\nThe second group are all cars.\n</reasoning>\n<answer>\n<group>apple, orange, banana, pear</group>\n<group>corolla, charger</group>\n</answer>\n\n# Example\n\nUser: dog, cat, red, white,\nAssistant: <reasoning>\nThe first group are all animals.\nThe second group are all colors.\n</reasoning>\n<answer>\n<group>dog, cat</group>\n<group>red, white</group>\n</answer>\n',
  'role': 'system'},
 {'content': 'hawking, match, genius, feynman, crikey, moot, flair, barbie, west, oppenheimer, einstein, faculty, gift, bush, mate, selling',
  'role': 'user'},
 {'role': 'assistant',
  'con